In [1]:
import random
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import csv
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from matplotlib import pyplot as plt
import datetime
import time
import lightgbm
import spacy
nlp_fr = spacy.load('fr_core_news_md', disable=["tagger", "parser", "ner", "textcat"])
nlp_en = spacy.load('en_core_web_md', disable=["tagger", "parser", "ner", "textcat"])
from tqdm import tqdm
import os
import networkx as nx
import pickle
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
import sklearn.feature_extraction.text as fe

Using TensorFlow backend.


In [3]:
with open(r"training.txt", "r") as f:
    reader = csv.reader(f)
    training  = list(reader)
    
training = [element[0].split(" ") for element in training]
training = pd.DataFrame(training, columns=['Node1', 'Node2', 'Link'])
print("Training examples shape: {}".format(training.shape))

with open(r"testing.txt", "r") as f:
    reader = csv.reader(f)
    testing  = list(reader)

testing = [element[0].split(" ") for element in testing]
testing = pd.DataFrame(testing, columns=['Node1', 'Node2'])
print("Testing examples shape: {}".format(testing.shape))

Training examples shape: (453797, 3)
Testing examples shape: (113450, 2)


In [4]:
'''
uncomment lines for reduced corpus with stopword removal. In future integrate stemmer here, multi-language
'''
NODE_INFO_DIRECTORY = r"node_information/text/"

corpus_path = r"pickles/corpus.PICKLE" 
ids_path = r"pickles/IDs.PICKLE"
if os.path.exists(corpus_path):
    with open(corpus_path, 'rb') as f:
        corpus = pickle.load(f)
    f.close()
    with open(ids_path, 'rb') as f:
        ids = pickle.load(f)
    f.close()
else:
    corpus = []
    ids = []
    for filename in tqdm(os.listdir(NODE_INFO_DIRECTORY)):
        with open(NODE_INFO_DIRECTORY + filename, 'r', encoding='UTF-8', errors='ignore') as f:
            doc_string = []
            for line in f:
                [doc_string.append(token) for token in line.lower().strip().split(" ")]
            corpus.append(' '.join(doc_string))
            ids.append(filename[:-4])
    with open(corpus_path, '+wb') as f:
        pickle.dump(corpus, f)
    f.close()
    with open(ids_path, '+wb') as f:
        pickle.dump(ids, f)
    f.close() 

# in order of alphabetical text information i.e. 0, 1, 10, 100
node_info = pd.DataFrame({'ID': ids, 'Corpus': corpus})
node_info_ID = node_info.set_index(['ID'])
print("Training node info shape: {}".format(node_info.shape))

Training node info shape: (33226, 2)


In [5]:
keep_indices = random.sample(range(len(training)), k=int(round(len(training)*0.05)))
data_train_val = training.iloc[keep_indices]
data_train = training.loc[~training.index.isin(keep_indices)]

In [6]:
linked_nodes = data_train.loc[data_train['Link']=='1']
linked_nodes = linked_nodes[['Node1', 'Node2']]
linked_nodes.to_csv('linked_nodes.txt', sep=' ', index=False, header=False)

In [7]:
G=nx.read_edgelist('linked_nodes.txt', create_using=nx.Graph(), nodetype = str)

In [ ]:
'''
for i in tqdm(list(node_info.index)):
    if node_info.loc[i]['ID'] in G:
        G.node[node_info.loc[i]['ID']]['T'] = node_info.loc[i]['Corpus']
'''

## Training

In [ ]:
katz_nd = nx.katz_centrality_numpy(G, alpha=0.1, beta=1.0)

In [ ]:
def katz_centrality(index, dataset):
    node1 = dataset['Node1'][index]
    node2 = dataset['Node2'][index]
    if (node1 in katz_nd.keys()) and (node2 in katz_nd.keys()): 
        katz = (katz_nd[node1] + katz_nd[node2])/2
        return katz
    else: 
        return 0

perf_val_data['page_rank']=list(map(lambda i:katz_centrality(i, training), tqdm(perf_val_data['original_index'])))
perf_test_data['page_rank']=list(map(lambda i:katz_centrality(i, testing), tqdm(perf_test_data['original_index'])))

In [ ]:
#switch to fit_transform for tfidf

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans, MiniBatchKMeans

print("Performing dimensionality reduction using LSA")
svd = TruncatedSVD(800)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(corpus_tfidf_matrix)

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
    int(explained_variance * 100)))


In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics

# #############################################################################
# Do the actual clustering

if opts.minibatch:
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000, verbose=opts.verbose)
else:
    km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=opts.verbose)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

print()

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
n_k = 1000
print("Extracting %d best features by a chi-squared test" %
      n_k)
ch2 = SelectKBest(chi2, k=n_k)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)

In [9]:
import stellargraph

In [8]:
import networkx as nx
from node2vec import Node2Vec

In [9]:
# Create a graph
graph = G

In [14]:
# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=1)  # Use temp_folder for big graphs




Computing transition probabilities:   0%|                                                    | 0/33125 [00:00<?, ?it/s]


Computing transition probabilities:   0%|                                          | 1/33125 [00:00<2:46:43,  3.31it/s]


Computing transition probabilities:   0%|                                          | 3/33125 [00:00<2:42:54,  3.39it/s]


Computing transition probabilities:   0%|                                          | 5/33125 [00:01<2:07:21,  4.33it/s]


Computing transition probabilities:   0%|                                          | 7/33125 [00:01<1:39:14,  5.56it/s]


Computing transition probabilities:   0%|                                          | 9/33125 [00:01<1:18:39,  7.02it/s]


Computing transition probabilities:   0%|                                         | 11/33125 [00:01<1:09:17,  7.96it/s]


Computing transition probabilities:   0%|                                           | 13/33125 [00:01<59:50,  9.22it/s]


Computing transition 

Computing transition probabilities:   0%|▏                                         | 128/33125 [00:18<57:02,  9.64it/s]


Computing transition probabilities:   0%|▏                                       | 130/33125 [00:18<1:15:09,  7.32it/s]


Computing transition probabilities:   0%|▏                                       | 131/33125 [00:18<1:10:54,  7.75it/s]


Computing transition probabilities:   0%|▏                                       | 132/33125 [00:18<1:36:39,  5.69it/s]


Computing transition probabilities:   0%|▏                                       | 133/33125 [00:19<1:42:47,  5.35it/s]


Computing transition probabilities:   0%|▏                                       | 134/33125 [00:19<1:42:28,  5.37it/s]


Computing transition probabilities:   0%|▏                                       | 136/33125 [00:19<1:20:18,  6.85it/s]


Computing transition probabilities:   0%|▏                                       | 138/33125 [00:19<1:15:31,  7.28it/s]


Computing transition pro

Computing transition probabilities:   1%|▎                                         | 284/33125 [00:33<52:52, 10.35it/s]


Computing transition probabilities:   1%|▎                                         | 286/33125 [00:33<58:21,  9.38it/s]


Computing transition probabilities:   1%|▎                                         | 288/33125 [00:33<53:09, 10.30it/s]


Computing transition probabilities:   1%|▎                                         | 290/33125 [00:33<51:49, 10.56it/s]


Computing transition probabilities:   1%|▎                                         | 293/33125 [00:34<55:05,  9.93it/s]


Computing transition probabilities:   1%|▎                                         | 295/33125 [00:34<48:05, 11.38it/s]


Computing transition probabilities:   1%|▍                                         | 297/33125 [00:34<52:12, 10.48it/s]


Computing transition probabilities:   1%|▍                                         | 299/33125 [00:34<55:00,  9.95it/s]


Computing transition pro

Computing transition probabilities:   1%|▌                                         | 462/33125 [00:47<47:03, 11.57it/s]


Computing transition probabilities:   1%|▌                                         | 465/33125 [00:48<42:57, 12.67it/s]


Computing transition probabilities:   1%|▌                                         | 470/33125 [00:48<34:49, 15.63it/s]


Computing transition probabilities:   1%|▌                                         | 473/33125 [00:48<46:02, 11.82it/s]


Computing transition probabilities:   1%|▌                                         | 477/33125 [00:48<37:24, 14.55it/s]


Computing transition probabilities:   1%|▌                                         | 480/33125 [00:49<51:50, 10.49it/s]


Computing transition probabilities:   1%|▌                                         | 482/33125 [00:49<47:02, 11.56it/s]


Computing transition probabilities:   1%|▌                                         | 484/33125 [00:49<50:31, 10.77it/s]


Computing transition pro

Computing transition probabilities:   2%|▊                                       | 629/33125 [01:06<4:23:14,  2.06it/s]


Computing transition probabilities:   2%|▊                                       | 631/33125 [01:06<3:16:02,  2.76it/s]


Computing transition probabilities:   2%|▊                                       | 633/33125 [01:06<2:40:08,  3.38it/s]


Computing transition probabilities:   2%|▊                                       | 635/33125 [01:07<2:15:24,  4.00it/s]


Computing transition probabilities:   2%|▊                                       | 638/33125 [01:07<1:45:16,  5.14it/s]


Computing transition probabilities:   2%|▊                                       | 641/33125 [01:07<1:23:45,  6.46it/s]


Computing transition probabilities:   2%|▊                                       | 643/33125 [01:07<1:10:58,  7.63it/s]


Computing transition probabilities:   2%|▊                                       | 645/33125 [01:07<1:02:10,  8.71it/s]


Computing transition pro

...

Computing transition probabilities:  99%|███████████████████████████████████████▍| 32698/33125 [22:02<00:12, 34.19it/s]


Computing transition probabilities:  99%|███████████████████████████████████████▍| 32704/33125 [22:02<00:12, 34.10it/s]


Computing transition probabilities:  99%|███████████████████████████████████████▍| 32710/33125 [22:02<00:10, 38.42it/s]


Computing transition probabilities:  99%|███████████████████████████████████████▌| 32715/33125 [22:03<00:09, 41.09it/s]


Computing transition probabilities:  99%|███████████████████████████████████████▌| 32722/33125 [22:03<00:08, 44.91it/s]


Computing transition probabilities:  99%|███████████████████████████████████████▌| 32728/33125 [22:03<00:12, 32.86it/s]


Computing transition probabilities:  99%|███████████████████████████████████████▌| 32733/33125 [22:03<00:10, 36.07it/s]


Computing transition probabilities:  99%|███████████████████████████████████████▌| 32738/33125 [22:03<00:11, 34.55it/s]


Computing transition pro

Computing transition probabilities: 100%|███████████████████████████████████████▉| 33071/33125 [22:12<00:01, 33.62it/s]


Computing transition probabilities: 100%|███████████████████████████████████████▉| 33075/33125 [22:12<00:01, 33.37it/s]


Computing transition probabilities: 100%|███████████████████████████████████████▉| 33083/33125 [22:13<00:01, 38.02it/s]


Computing transition probabilities: 100%|███████████████████████████████████████▉| 33096/33125 [22:13<00:00, 45.91it/s]


Computing transition probabilities: 100%|███████████████████████████████████████▉| 33102/33125 [22:13<00:00, 45.05it/s]


Computing transition probabilities: 100%|███████████████████████████████████████▉| 33108/33125 [22:13<00:00, 40.92it/s]


Computing transition probabilities: 100%|███████████████████████████████████████▉| 33113/33125 [22:13<00:00, 36.38it/s]


Computing transition probabilities: 100%|████████████████████████████████████████| 33125/33125 [22:13<00:00, 24.83it/s]



Generating walks (CPU: 

Generating walks (CPU: 1):  30%|██████████████▋                                  | 60/200 [2:35:25<6:08:00, 157.72s/it]


Generating walks (CPU: 1):  30%|██████████████▉                                  | 61/200 [2:37:58<6:02:11, 156.34s/it]


Generating walks (CPU: 1):  31%|███████████████▏                                 | 62/200 [2:40:33<5:58:21, 155.80s/it]


Generating walks (CPU: 1):  32%|███████████████▍                                 | 63/200 [2:43:23<6:05:46, 160.20s/it]


Generating walks (CPU: 1):  32%|███████████████▋                                 | 64/200 [2:46:17<6:12:28, 164.33s/it]


Generating walks (CPU: 1):  32%|███████████████▉                                 | 65/200 [2:48:56<6:06:02, 162.69s/it]


Generating walks (CPU: 1):  33%|████████████████▏                                | 66/200 [2:51:32<5:59:00, 160.75s/it]


Generating walks (CPU: 1):  34%|████████████████▍                                | 67/200 [2:54:05<5:51:02, 158.36s/it]


Generating walks (CPU: 1

Generating walks (CPU: 1):  64%|██████████████████████████████▍                 | 127/200 [5:27:57<2:35:24, 127.73s/it]


Generating walks (CPU: 1):  64%|██████████████████████████████▋                 | 128/200 [5:30:08<2:34:26, 128.70s/it]


Generating walks (CPU: 1):  64%|██████████████████████████████▉                 | 129/200 [5:32:19<2:32:59, 129.29s/it]


Generating walks (CPU: 1):  65%|███████████████████████████████▏                | 130/200 [5:34:26<2:30:04, 128.63s/it]


Generating walks (CPU: 1):  66%|███████████████████████████████▍                | 131/200 [5:36:35<2:27:54, 128.62s/it]


Generating walks (CPU: 1):  66%|███████████████████████████████▋                | 132/200 [5:38:41<2:24:53, 127.85s/it]


Generating walks (CPU: 1):  66%|███████████████████████████████▉                | 133/200 [5:40:47<2:22:11, 127.34s/it]


Generating walks (CPU: 1):  67%|████████████████████████████████▏               | 134/200 [5:42:56<2:20:38, 127.86s/it]


Generating walks (CPU: 1

Generating walks (CPU: 1):  97%|████████████████████████████████████████████████▌ | 194/200 [8:04:13<13:13, 132.25s/it]


Generating walks (CPU: 1):  98%|████████████████████████████████████████████████▊ | 195/200 [8:06:28<11:05, 133.00s/it]


Generating walks (CPU: 1):  98%|█████████████████████████████████████████████████ | 196/200 [8:08:51<09:03, 135.86s/it]


Generating walks (CPU: 1):  98%|█████████████████████████████████████████████████▎| 197/200 [8:11:10<06:50, 136.88s/it]


Generating walks (CPU: 1):  99%|█████████████████████████████████████████████████▌| 198/200 [8:13:18<04:28, 134.24s/it]


Generating walks (CPU: 1): 100%|█████████████████████████████████████████████████▊| 199/200 [8:15:28<02:13, 133.03s/it]


Generating walks (CPU: 1): 100%|██████████████████████████████████████████████████| 200/200 [8:19:51<00:00, 149.96s/it]


In [15]:
# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed

In [24]:
# Save embeddings for later use
model.wv.save_word2vec_format('pickles/EMBEDDING_FILENAME.embed')

In [25]:
# Save model for later use
model.save('pickles/EMBEDDING_MODEL_FILENAME.model')

In [36]:
# Embed edges using Hadamard method
from node2vec.edges import HadamardEmbedder
edges_embs = HadamardEmbedder(keyed_vectors=model.wv)


Generating edge features:   0%|                                      | 323146/548649375.0 [00:21<1:38:46, 92520.67it/s]

In [50]:
embed_size = len(edges_embs[('0', '1')])
df = pd.DataFrame(0, index=np.arange(len(training)), columns=range(embed_size))
j = []
for i in tqdm(range(len(training)), position=0, leave=True):
    try:
        df.loc[i] = edges_embs[(training.loc[i]['Node1'], training.loc[i]['Node2'])]
    except:
        df.loc[i] = np.zeros(embed_size)

100%|█████████████████████████████████████████████████████████████████████████| 453797/453797 [17:18<00:00, 436.97it/s]


In [55]:
X = df
y = training['Link']
y = list(map(lambda i: int(i), y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)

In [56]:
lgbm = lightgbm.LGBMClassifier()
model_lgbm = lgbm.fit(X_train, y_train)
predictions = model_lgbm.predict(X_test)
print(f1_score(y_test, np.round(predictions)))

0.967147972738824


In [74]:
embed_size = len(edges_embs[('0', '1')])
df_test = pd.DataFrame(0, index=np.arange(len(testing)), columns=range(embed_size))
j = []
for i in tqdm(range(len(testing)), position=0, leave=True):
    try:
        df_test.loc[i] = edges_embs[(testing.loc[i]['Node1'], testing.loc[i]['Node2'])]
    except:
        df_test.loc[i] = np.zeros(embed_size)

100%|█████████████████████████████████████████████████████████████████████████| 113450/113450 [03:42<00:00, 511.03it/s]
